<a href="https://colab.research.google.com/github/FFI-Vietnam/camtrap-tools/blob/main/Wildlife%20Insights/bulk-upload/04_collect-WI-taxon-database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
"""
This script is used for cleaning the metadata spreadsheet generated by ExifTool
After runnning this script, 'data cleaning' folder is created

data cleaning
    |__ 4.1.  csv
    
"""

"\nThis script is used for cleaning the metadata spreadsheet generated by ExifTool\nAfter runnning this script, 'data cleaning' folder is created\n\ndata cleaning\n    |__ 4.1.  csv\n    \n"

In [3]:
import pandas as pd
import numpy as np
import os
import json
import requests

# mount with Google Drive to read and save file
from google.colab import drive
drive.mount('/content/drive')

# specifies Colab directories and file names
root = '/content/drive/'

dataset_folder = 'My Drive/FFI/Wildlife Insights Bulk Upload Test/bulk-upload_template-autofill/dataset'
contain_folder = 'My Drive/FFI/Wildlife Insights Bulk Upload Test/bulk-upload_template-autofill/data cleaning'

WIdatabase_file_name = "4.1_collect_WI-taxon-database.csv"

# useful functions
def read_csv_Google_drive(root, contain_folder, file_name):
  """
  function to read a csv file from Google Drive
  param examples:
    root = '/content/drive/'
    contain_folder = 'My Drive/FFI/dataset'
    file_name = 'image_metadata(2020-06-26)_full.csv'
  """
  file_path = os.path.join(root, contain_folder, file_name)
  return pd.read_csv(file_path)

def save_csv_Google_drive(df, root, contain_folder, file_name):
  """
  function to save a csv file to Google Drive
  param examples:
    root = '/content/drive/'
    contain_folder = 'My Drive/FFI/dataset'
    file_name = 'image_metadata(2020-06-26)_full.csv'
  """
  # save file to Colab runtime storage (will be deleted when this notebook is closed)
  df.to_csv('dataframe.csv', index=False)

  # save file back to Google Drive for permanent storage
  folder_path = os.path.join(root, contain_folder)
  file_path = os.path.join(root, contain_folder, file_name)
  try:
    os.makedirs(folder_path)
  except:
    pass

  with open('dataframe.csv', 'r') as f:
    df_file = f.read()

  with open(file_path, 'w') as f:
    f.write(df_file)

  print(f'File is saved to {file_name} in Google Drive at {file_path}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# 4.1) 

# download WI taxon id database
URL = "https://api.wildlifeinsights.org/api/v1/taxonomy?fields=class,order,family,genus,species,authority,taxonomyType,uniqueIdentifier,commonNameEnglish&page[size]=30000"
response = requests.get(URL)

# temporarily save the database to a json file
with open('taxonomy.json', 'w') as outfile:
    json.dump(response.json()["data"], outfile)

taxonomy_df = pd.read_json(r'taxonomy.json')

In [5]:
# find species who is not in WI taxon db

df = read_csv_Google_drive(root, contain_folder, "1.4_clean_metadata_remove-no-animal-image.csv")

# list of species
species_list = df['species_common_name'].unique()

unfound = []
for s in species_list:
  if (s not in taxonomy_df['commonNameEnglish'].to_list()):
    unfound.append(s)

print("There are",len(unfound),"unfound species")
unfound

There are 54 unfound species


['Yellow-breasted Magpie',
 'Emerald Dove',
 'Racket-tailed Treepie',
 'Ochraceous Bulbul',
 'Murid',
 'Ferret-badger',
 'Hill Blue Flycatcher',
 'Moth',
 'White-throated Fantail',
 'Asian Red-cheeked Squirrel',
 'Leopoldamys',
 'Asian Palm Civet',
 'Indochinese Wren-babbler',
 'Squirrel',
 'Banded Kingfisher',
 'Rhesus Macaque',
 'Wild Pig',
 'Crested Serpent Eagle',
 'Annamite Dark Muntjac',
 'Flying Insect',
 'Snake',
 'Black-throated Laughingthrush',
 'Asian Black Bear',
 'White-Crowned Forktail',
 'Macaque',
 'Animal',
 'Scaly Thrush',
 'Impressed Tortoise',
 'Maxomys',
 'Banded Krait',
 'Small Asian Mongoose',
 'Oriental Magpie Robin',
 'Muntjac',
 'Asian Water Monitor',
 'Japanese Robin',
 'Particolored Flying Squirrel',
 'Shrew',
 'Butterfly',
 'Brownish-flanked Bush Warbler',
 'White-tailed Robin',
 'Flying squirrel',
 'Streaked Wren Babbler',
 'Lesser Mouse-deer',
 'Domestic Buffalo',
 'Green magpie',
 'Pale-footed Bush Warbler',
 'Greater Racket-tailed Drongo',
 'Eastern Str

In [6]:
# (optional) create template match_unfound_species matching for the below cell
for i in unfound:
  print('{:60}'.format('match_unfound_species["'+i+'"]') + '= {"commonNameEnglish":""}')

match_unfound_species["Yellow-breasted Magpie"]             = {"commonNameEnglish":""}
match_unfound_species["Emerald Dove"]                       = {"commonNameEnglish":""}
match_unfound_species["Racket-tailed Treepie"]              = {"commonNameEnglish":""}
match_unfound_species["Ochraceous Bulbul"]                  = {"commonNameEnglish":""}
match_unfound_species["Murid"]                              = {"commonNameEnglish":""}
match_unfound_species["Ferret-badger"]                      = {"commonNameEnglish":""}
match_unfound_species["Hill Blue Flycatcher"]               = {"commonNameEnglish":""}
match_unfound_species["Moth"]                               = {"commonNameEnglish":""}
match_unfound_species["White-throated Fantail"]             = {"commonNameEnglish":""}
match_unfound_species["Asian Red-cheeked Squirrel"]         = {"commonNameEnglish":""}
match_unfound_species["Leopoldamys"]                        = {"commonNameEnglish":""}
match_unfound_species["Asian Palm Civet"]  

Use these query to find common species name

In [7]:
taxonomy_df[(taxonomy_df['genus'] == 'Homo') & (taxonomy_df['species'] == 'sapiens')]

,id,class,order,family,genus,species,authority,commonNameEnglish,taxonomyType,uniqueIdentifier
17499,2021525,Mammalia,Primates,Hominidae,Homo,sapiens,None,Human-Camera Trapper,biological,066c4187-4a03-4531-9f62-ac585ad0a138
17500,2021526,Mammalia,Primates,Hominidae,Homo,sapiens,None,Human-Hunter,biological,f5ec47a5-e7a4-4158-a0b4-a6481ae3f06f
17501,2021527,Mammalia,Primates,Hominidae,Homo,sapiens,None,Human-Researcher,biological,e6f88ed1-c305-4e94-b53d-8eec1b1d980a
17502,2021528,Mammalia,Primates,Hominidae,Homo,sapiens,None,Human-Tourist,biological,795880f2-2677-41bf-be13-72c58056f08d
17503,2021529,Mammalia,Primates,Hominidae,Homo,sapiens,None,Human-Resident,biological,128d0fac-f96d-4a98-af18-2903dbbe1568
17545,2021530,Mammalia,Primates,Hominidae,Homo,sapiens,None,Human-Park Ranger,biological,0ffb7a18-d6df-454c-a014-c99f52ec22a1
17546,2021531,Mammalia,Primates,Hominidae,Homo,sapiens,None,Human-Maintenance crew,biological,41ebc3f1-ecc7-497c-acd4-6b35623a9465
17547,2021532,Mammalia,Primates,Hominidae,Homo,sapiens,None,Human-Pedestrian,biological,0de272bf-5eab-4198-a888-3a6e92f1460d
21540,2002045,Mammalia,Primates,Hominidae,Homo,sapiens,"Linnaeus, 1758",Human,biological,990ae9dd-7a59-4344-afcb-1b7b21368000


In [8]:
taxonomy_df[taxonomy_df['species'] == 'owstoni']

,id,class,order,family,genus,species,authority,commonNameEnglish,taxonomyType,uniqueIdentifier
5708,2007569,Aves,Gruiformes,Rallidae,Hypotaenidia,owstoni,"Rothschild, 1895",Guam Rail,biological,8702d3ea-c6a6-4327-99bd-e033f1773d60
7443,2009882,Aves,Piciformes,Picidae,Dendrocopos,owstoni,"Ogawa, 1905",Amami Woodpecker,biological,4eb83928-a7a9-4373-8e19-b872bdc10d7d
11425,2015369,Aves,Passeriformes,Muscicapidae,Ficedula,owstoni,"Bangs, 1901",Ryukyu Flycatcher,biological,07095ce5-0b03-4851-854e-a09437b8481f
13148,2000801,Mammalia,Carnivora,Viverridae,Chrotogale,owstoni,"Thomas, 1912",Owston's Civet,biological,ba799fc7-6aad-455e-87a8-1f09ad5c71e4
16388,2013174,Aves,Passeriformes,Paridae,Sittiparus,owstoni,"Ijima, 1893",Izu Tit,biological,c5085bc0-c596-48f0-8a14-7ee1991a38a9


In [9]:
taxonomy_df[(taxonomy_df['genus'] == 'Homo')]

,id,class,order,family,genus,species,authority,commonNameEnglish,taxonomyType,uniqueIdentifier
17499,2021525,Mammalia,Primates,Hominidae,Homo,sapiens,None,Human-Camera Trapper,biological,066c4187-4a03-4531-9f62-ac585ad0a138
17500,2021526,Mammalia,Primates,Hominidae,Homo,sapiens,None,Human-Hunter,biological,f5ec47a5-e7a4-4158-a0b4-a6481ae3f06f
17501,2021527,Mammalia,Primates,Hominidae,Homo,sapiens,None,Human-Researcher,biological,e6f88ed1-c305-4e94-b53d-8eec1b1d980a
17502,2021528,Mammalia,Primates,Hominidae,Homo,sapiens,None,Human-Tourist,biological,795880f2-2677-41bf-be13-72c58056f08d
17503,2021529,Mammalia,Primates,Hominidae,Homo,sapiens,None,Human-Resident,biological,128d0fac-f96d-4a98-af18-2903dbbe1568
17545,2021530,Mammalia,Primates,Hominidae,Homo,sapiens,None,Human-Park Ranger,biological,0ffb7a18-d6df-454c-a014-c99f52ec22a1
17546,2021531,Mammalia,Primates,Hominidae,Homo,sapiens,None,Human-Maintenance crew,biological,41ebc3f1-ecc7-497c-acd4-6b35623a9465
17547,2021532,Mammalia,Primates,Hominidae,Homo,sapiens,None,Human-Pedestrian,biological,0de272bf-5eab-4198-a888-3a6e92f1460d
17548,2020796,Mammalia,Primates,Hominidae,Homo,None,None,Homo Species,biological,e3954aac-8c80-4308-9ca4-0f35b079fe3b
21540,2002045,Mammalia,Primates,Hominidae,Homo,sapiens,"Linnaeus, 1758",Human,biological,990ae9dd-7a59-4344-afcb-1b7b21368000


In [10]:
taxonomy_df[(taxonomy_df['family'] == 'Viverridae')]

,id,class,order,family,genus,species,authority,commonNameEnglish,taxonomyType,uniqueIdentifier
202,2000266,Mammalia,Carnivora,Viverridae,Arctictis,binturong,"Raffles, 1821",Binturong,biological,b310c664-bb99-487b-9c54-fb0935165d29
211,2000277,Mammalia,Carnivora,Viverridae,Arctogalidia,trivirgata,"Gray, 1832",Small-toothed Palm Civet,biological,ce0a56c1-1e05-44cc-b671-cfe932e8d5e4
614,2000816,Mammalia,Carnivora,Viverridae,Civettictis,civetta,"Schreber, 1776",African Civet,biological,127278f5-8bf3-4f85-ba19-99e097f17f46
893,2001191,Mammalia,Carnivora,Viverridae,Cynogale,bennettii,"Gray, 1837",Otter Civet,biological,e05a16ba-2438-4b1a-b0ed-cc1900a794be
1006,2001344,Mammalia,Carnivora,Viverridae,Diplogale,hosei,"Thomas, 1892",Hose's Civet,biological,96ef11b4-e905-4ab1-b0c2-583e773fd71e
1272,2001694,Mammalia,Carnivora,Viverridae,Genetta,angolensis,"Bocage, 1882",Miombo Genet,biological,37345f1b-413a-464f-9f2c-8fdb97c429ee
1273,2001695,Mammalia,Carnivora,Viverridae,Genetta,bourloni,"Gaubert, 2003",Bourlon's Genet,biological,958aee5e-f6e5-492e-9e14-41b55b637296
1274,2001696,Mammalia,Carnivora,Viverridae,Genetta,cristata,"Hayman in Sanborn, 1940",Crested Genet,biological,8c4f8190-6972-48f4-af4c-8c43182cb5b0
1275,2001698,Mammalia,Carnivora,Viverridae,Genetta,johnstoni,"Pocock, 1908",Johnston's Genet,biological,a8fda858-5493-430e-8c73-516598387b5e
1276,2001699,Mammalia,Carnivora,Viverridae,Genetta,maculata,"Gray, 1830",Large-spotted Genet,biological,e33e9444-e982-4c05-a7d1-cce2972f5f8e


In [11]:
taxonomy_df[(taxonomy_df['order'] == 'Carnivora')]

,id,class,order,family,genus,species,authority,commonNameEnglish,taxonomyType,uniqueIdentifier
16,2000023,Mammalia,Carnivora,Felidae,Acinonyx,jubatus,"Schreber, 1775",Cheetah,biological,5a9b1344-2dd5-4162-9d85-77ee4648ad9c
49,2000067,Mammalia,Carnivora,Ursidae,Ailuropoda,melanoleuca,"David, 1869",Giant Panda,biological,85662682-67c1-4ecb-ba05-ba12e2df6b65
51,2000070,Mammalia,Carnivora,Ailuridae,Ailurus,fulgens,"F.G. Cuvier, 1825",Red Panda,biological,7bef348d-37cf-4b5b-bf91-e4fe2dfc483c
156,2000206,Mammalia,Carnivora,Mustelidae,Aonyx,capensis,"Schinz, 1821",African Clawless Otter,biological,0d824975-afe6-4463-9805-37a6ba260d2a
157,2000208,Mammalia,Carnivora,Mustelidae,Aonyx,congicus,"Lönnberg, 1910",Congo Clawless Otter,biological,7a7cdfdd-5fd7-4e1a-bb5c-9b82aff735b6
...,...,...,...,...,...,...,...,...,...,...
21501,2021469,Mammalia,Carnivora,Ursidae,None,None,None,Bear Family,biological,ec1a70f4-41c0-4aba-9150-292fb2b7a324
21527,2021243,Mammalia,Carnivora,Canidae,None,None,None,Canine Family,biological,3184697f-51ad-4608-9a28-9edb5500159c
21528,2021142,Mammalia,Carnivora,Felidae,None,None,None,Cat Family,biological,df8514b0-10a5-411f-8ed6-0f415e8153a3
21597,2021548,Mammalia,Carnivora,Canidae,Canis,familiaris,None,Domestic Dog,biological,3d80f1d6-b1df-4966-9ff4-94053c7a902a


In [12]:
taxonomy_df[(taxonomy_df['class'] == 'Mammalia')]

,id,class,order,family,genus,species,authority,commonNameEnglish,taxonomyType,uniqueIdentifier
0,2000003,Mammalia,Rodentia,Abrocomidae,Abrocoma,bennettii,"Waterhouse, 1837",Bennett's Chinchilla Rat,biological,7a6c93a5-bdf7-4182-82f9-7a67d23f7fe1
1,2000004,Mammalia,Rodentia,Abrocomidae,Abrocoma,boliviensis,"Glanz & Anderson, 1990",Bolivian Chinchilla Rat,biological,25992d0a-5814-48ee-a2f3-e26862dbe0db
2,2000005,Mammalia,Rodentia,Abrocomidae,Abrocoma,budini,"Thomas, 1920",Budin's Chinchilla Rat,biological,b2307167-7f7b-4349-8f3d-6c1786720290
3,2000006,Mammalia,Rodentia,Abrocomidae,Abrocoma,cinerea,"Thomas, 1919",Ashy Chinchilla Rat,biological,8bb0e53f-a1f8-4387-88fe-0eabf00d06f2
4,2000008,Mammalia,Rodentia,Abrocomidae,Abrocoma,shistacea,"Thomas, 1921",Sierra Del Tontal Chinchilla Rat,biological,cd6e2617-a488-4647-a9d5-b67f7a18af60
...,...,...,...,...,...,...,...,...,...,...
21704,2021730,Mammalia,Carnivora,Felidae,Felis,silvestris lybica,NA,African Wild Cat,biological,d620e25c-eed1-40f1-8101-6d3b33f3a723
21705,2021731,Mammalia,Artiodactyla,Bovidae,Alcelaphus,buselaphus caama,"Geoffroy Saint-Hilaire, 1803",Red Hartebeest,biological,7bc9c5de-302c-409e-bcae-fb70dcb23015
21706,2021732,Mammalia,Primates,Cercopithecidae,Cercopithecus,albogularis,"Sykes, 1831",Samango Monkey,biological,2a6bedf2-0d87-4d1e-a59a-e2466d53f2a7
21707,2021733,Mammalia,Rodentia,Sciuridae,Geosciurus,inauris,"Zimmerman, 1780",Cape Ground Squirrel,biological,25daef26-08e3-478a-9bd5-134580cf47b1


In [13]:
# match unfound species name with corresponding name in WI taxon id

match_unfound_species = {}
# match_unfound_species["Hill Blue Flycatcher"]
# match_unfound_species["Indochinese Wren-babbler"] = {"Taxonomy":"Gypsophila brevicaudata"}
# match_unfound_species["Black-throated Laughingthrush"] = {"Taxonomy":"Pterorhinus chinensis"}
# match_unfound_species["Banded Krait"] = {"Taxonomy":"Bungarus fasciatus"}
# match_unfound_species["Impressed Tortoise"] = {"Taxonomy":"Manouria impressa"}
# match_unfound_species["Small Asian Mongoose"] = {"genus":"Urva", "species":"javanica"}
# match_unfound_species["Natalia's Agama"] = {"commonNameEnglish":""}

# match_unfound_species["Flying Insect"] = {"commonNameEnglish":""}
# match_unfound_species["Animal"] = {"commonNameEnglish":""}

match_unfound_species["Streaked Wren Babbler"]              = {"commonNameEnglish":"Streaked Wren-babbler"}
match_unfound_species["Asian Water Monitor"]                = {"commonNameEnglish":"Common Water Monitor"}
match_unfound_species["Asian Red-cheeked Squirrel"]         = {"commonNameEnglish":"Red-cheeked Squirrel"}
match_unfound_species["Muntjac"]                            = {"commonNameEnglish":"Muntiacus Species"}
match_unfound_species["Asian Black Bear"]                   = {"commonNameEnglish":"Asiatic Black Bear"}
match_unfound_species["Ferret-badger"]                      = {"commonNameEnglish":"Melogale Species"}
match_unfound_species["Rhesus Macaque"]                     = {"commonNameEnglish":"Rhesus Monkey"}
match_unfound_species["Murid"]                              = {"commonNameEnglish":"Muridae Family"}
match_unfound_species["Maxomys"]                            = {"commonNameEnglish":"Muridae Family"}
match_unfound_species["Leopoldamys"]                        = {"commonNameEnglish":"Muridae Family"}
match_unfound_species["Wild Pig"]                           = {"commonNameEnglish":"Wild Boar"}
match_unfound_species["Crested Serpent Eagle"]              = {"commonNameEnglish":"Crested Serpent-Eagle"}
match_unfound_species["Yellow-breasted Magpie"]             = {"commonNameEnglish":"Indochinese Green Magpie"}
match_unfound_species["Macaque"]                            = {"commonNameEnglish":"Macaca Species"}
match_unfound_species["Snake"]                              = {"commonNameEnglish":"Lizards and Snakes"}
match_unfound_species["Annamite Dark Muntjac"]              = {"commonNameEnglish":"Annamite Muntjac"}
match_unfound_species["White-Crowned Forktail"]             = {"commonNameEnglish":"White-crowned Forktail"}
match_unfound_species["Particolored Flying Squirrel"]       = {"commonNameEnglish":"African Linsang"}
match_unfound_species["Oriental Magpie Robin"]              = {"commonNameEnglish":"Oriental Magpie-Robin"}
match_unfound_species["Squirrel"]                           = {"commonNameEnglish":"Sciuridae Family"}
match_unfound_species["Butterfly"]                          = {"commonNameEnglish":"Butterflies and Moths"}
match_unfound_species["Moth"]                               = {"commonNameEnglish":"Butterflies and Moths"}
match_unfound_species["Asian Palm Civet"]                   = {"commonNameEnglish":"Common Palm Civet"}
match_unfound_species["Shrew"]                              = {"commonNameEnglish":"Soricidae Family"}
match_unfound_species["Racket-tailed Treepie"]              = {"commonNameEnglish":"Racquet-tailed Treepie"}
match_unfound_species["Emerald Dove"]                       = {"commonNameEnglish":"Common Emerald Dove"}
match_unfound_species["Domestic Buffalo"]                   = {"commonNameEnglish":"Bubalus Species"}
match_unfound_species["Malayan Night Heron"]                = {"commonNameEnglish":"Malay Night-heron"}
match_unfound_species["Lesser Mouse-deer"]                  = {"commonNameEnglish":"Lesser Oriental Chevrotain"}
match_unfound_species["Eastern Striped Squirrel"]           = {"commonNameEnglish":"Maritime Striped Squirrel"}
match_unfound_species["Green magpie"]                       = {"commonNameEnglish":"Common Green Magpie"}
match_unfound_species["Flying squirrel"]                    = {"commonNameEnglish":"Sciuridae Family"}
match_unfound_species["White-tailed Robin"]                 = {"commonNameEnglish":"White-tailed Blue Robin"}
match_unfound_species["Tickell's Blue Flycatcher"]          = {"commonNameEnglish":"Tickell's Blue-Flycatcher"}
match_unfound_species["Pale-footed Bush Warbler"]           = {"commonNameEnglish":"Pale-footed Bush-warbler"}  
match_unfound_species["Banded Kingfisher"]                  = {"genus":"Lacedo",     "species":"pulchella"}
match_unfound_species["Japanese Robin"]                     = {"genus":"Larvivora",  "species":"akahige"}
match_unfound_species["Ochraceous Bulbul"]                  = {"genus":"Alophoixus", "species":"ochraceus"}
match_unfound_species["Scaly Thrush"]                       = {"genus":"Zoothera",   "species":"dauma"}
match_unfound_species["White-throated Fantail"]             = {"genus":"Rhipidura",  "species":"albicollis"}
match_unfound_species["Brownish-flanked Bush Warbler"]      = {"genus":"Horornis",   "species":"fortipes"}
match_unfound_species["Greater Racquet-tailed Drongo"]      = {"genus":"Dicrurus",   "species":"paradiseus"}
match_unfound_species["Greater Racket-tailed Drongo"]       = {"genus":"Dicrurus",   "species":"paradiseus"}
match_unfound_species["Snowy-browed Flycatcher"]            = {"genus":"Ficedula",   "species":"hyperythra"}
match_unfound_species["Puff-throated Bulbul"]               = {"genus":"Alophoixus", "species":"pallidus"}

In [14]:
# rename some species
match_unfound_species["Chinese Serow"] = {"commonNameEnglish":"Sumatran Serow"}

In [18]:
# match species name with corresponding name in WI taxon id

match_table = taxonomy_df[taxonomy_df['commonNameEnglish'] == ""]
FFI_species_name = []

species_list = df['species_common_name'].dropna().unique()
# add Blank as a species
species_list = np.append(species_list, 'Blank')

for s in species_list:
  if (s in match_unfound_species): 
    FFI_species_name.append(s)
    # if the species has commonNameEnglish
    if 'commonNameEnglish' in match_unfound_species[s]:
      if taxonomy_df[taxonomy_df['commonNameEnglish'] == match_unfound_species[s]['commonNameEnglish']].empty:
        print('Cannot find', s)
        continue
      match_table = match_table.append(taxonomy_df[taxonomy_df['commonNameEnglish'] == match_unfound_species[s]['commonNameEnglish']])
    # if the species has genus
    else:
      if taxonomy_df[(taxonomy_df['genus'] == match_unfound_species[s]['genus']) &
                      (taxonomy_df['species'] == match_unfound_species[s]['species'])].empty:
        print('Cannot find', s)
        continue
      match_table = match_table.append(taxonomy_df[(taxonomy_df['genus'] == \
                                                      match_unfound_species[s]['genus']) & \
                                                      (taxonomy_df['species'] == \
                                                      match_unfound_species[s]['species'])])

  else:
    if s in taxonomy_df['commonNameEnglish'].to_list():
      FFI_species_name.append(s)
      match_table = match_table.append(taxonomy_df[taxonomy_df['commonNameEnglish'] == s])
    else:
      print(f"{s} not found")

match_table['FFI_species_name'] = FFI_species_name

# save match_table to Google Drive
save_csv_Google_drive(match_table, root, contain_folder, WIdatabase_file_name)
match_table.sample(7)

Hill Blue Flycatcher not found
Indochinese Wren-babbler not found
Flying Insect not found
Black-throated Laughingthrush not found
Animal not found
Impressed Tortoise not found
Banded Krait not found
Small Asian Mongoose not found
Natalia's Agama not found
File is saved to 4.1_collect_WI-taxon-database.csv in Google Drive at /content/drive/My Drive/FFI/Wildlife Insights Bulk Upload Test/bulk-upload_template-autofill/data cleaning/4.1_collect_WI-taxon-database.csv


,id,class,order,family,genus,species,authority,commonNameEnglish,taxonomyType,uniqueIdentifier,FFI_species_name
10792,2014516,Aves,Passeriformes,Leiotrichidae,Garrulax,monileger,"Riley, 1930",Lesser Necklaced Laughingthrush,biological,cfd634c7-a7b2-4fc2-b4c5-698aec23df3c,Lesser Necklaced Laughingthrush
4245,2005618,Mammalia,Carnivora,Viverridae,Viverricula,indica,"É. Geoffroy Saint-Hilaire, 1803",Small Indian Civet,biological,f68b2a88-9655-499b-b3c3-f9825ed3ba2a,Small Indian Civet
19869,2021113,Aves,None,None,None,None,None,Bird,biological,b1352069-a39c-4a84-a949-60044271c0c1,Bird
10729,2014432,Aves,Passeriformes,Pellorneidae,Trichastoma,tickelli,"Blyth, 1859",Buff-breasted Babbler,biological,bad6ab4c-cedb-4cf9-8a78-d6e6d20c97d9,Buff-breasted Babbler
16891,2015156,Aves,Passeriformes,Muscicapidae,Kittacincla,malabarica,"Scopoli, 1788",White-rumped Shama,biological,aa8579dc-5b25-4347-b1ac-31668aede87b,White-rumped Shama
21526,2021317,Mammalia,Rodentia,Sciuridae,None,None,None,Sciuridae Family,biological,e4d1e892-0e4b-475a-a8ac-b5c3502e0d55,Squirrel
9439,2012701,Aves,Passeriformes,Rhipiduridae,Rhipidura,albicollis,"Vieillot, 1818",None,biological,0f3f4083-15ca-4a13-96e2-32a5170e3a23,White-throated Fantail
